In [1]:
import os
import numpy as np
import cv2
import torch
from pathlib import Path
from termcolor import colored, cprint

from hloc import extract_features, match_features, localize_inloc, visualization, pairs_from_retrieval

In [2]:
print("GPU ON: ", torch.cuda.is_available())

images = Path("datasets/loc")
images_train = images / "train"
images_test = images / "test"

outputs = Path("outputs/loc_indoor/")
sfm_pairs = outputs / "pairs-netvlad.txt"

retrieval_conf = extract_features.confs["netvlad"]



GPU ON:  True


In [3]:
# build the database
retrieval_path_train = extract_features.main(retrieval_conf, images_train, outputs)
pairs_from_retrieval.main(retrieval_path_train, sfm_pairs, num_matched=5)
torch.cuda.empty_cache()

[2024/03/08 19:48:23 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2024/03/08 19:48:23 hloc INFO] Found 1205 images in root datasets/loc/train.
100%|██████████████████████████████████████████████████████████████████████████████████████████| 1205/1205 [02:37<00:00,  7.64it/s]
[2024/03/08 19:51:06 hloc INFO] Finished exporting features.
[2024/03/08 19:51:06 hloc INFO] Extracting image pairs from a retrieval database.
[2024/03/08 19:51:07 hloc INFO] Found 6025 pairs.


In [4]:
# add test images into traind dataset
retrieval_path_test = extract_features.main(retrieval_conf, images_test, outputs)
re = pairs_from_retrieval.main(retrieval_path_test, sfm_pairs, num_matched=5)
torch.cuda.empty_cache()

[2024/03/08 19:51:07 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2024/03/08 19:51:07 hloc INFO] Found 1 images in root datasets/loc/test.
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.24it/s]
[2024/03/08 19:51:12 hloc INFO] Finished exporting features.
[2024/03/08 19:51:12 hloc INFO] Extracting image pairs from a retrieval database.
[2024/03/08 19:51:13 hloc INFO] Found 6030 pairs.


# Above already can find the coarse pair

### below is going to find precise location & pose

In [5]:
# dense matching by superpoint
feature_conf = extract_features.confs["superpoint_inloc"]
matcher_conf = match_features.confs["superglue"]
# matcher_conf = match_features.confs["superpoint+lightglue"]

%time feature_path = extract_features.main(feature_conf, images, outputs)
cprint("Features extracted! ", "red")

match_path = match_features.main(
    matcher_conf, sfm_pairs, feature_conf["output"], outputs
)

[2024/03/08 19:51:13 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2024/03/08 19:51:14 hloc INFO] Found 1340 images in root datasets/loc.


Loaded SuperPoint model


100%|██████████████████████████████████████████████████████████████████████████████████████████| 1340/1340 [01:48<00:00, 12.30it/s]
[2024/03/08 19:53:02 hloc INFO] Finished exporting features.
[2024/03/08 19:53:02 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


CPU times: user 1min 50s, sys: 1.97 s, total: 1min 51s
Wall time: 1min 48s
Features extracted! 
Loaded SuperGlue model ("outdoor" weights)


  0%|                                                                                                     | 0/3705 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/shu/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/shu/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/shu/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/shu/My/HLOC_dev/hloc/match_features.py", line 127, in __getitem__
    grp = fd[name0]
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/home/shu/miniconda3/envs/pytorch/lib/python3.10/site-packages/h5py/_hl/group.py", line 357, in __